### dead code

In [ ]:
# loading_data_first_time = False
# if loading_data_first_time:
#     raise NotImplementedError("This is not implemented yet")
#     data = load_dataset(
#         "NeelNanda/c4-code-tokenized-2b", split="train", cache_dir="/workspace/cache/"
#     )
#     data.save_to_disk("/workspace/data/c4_code_tokenized_2b.hf")
#     data.set_format(type="torch", columns=["tokens"])
#     all_tokens = data["tokens"]
#     all_tokens.shape

#     all_tokens_reshaped = einops.rearrange(
#         all_tokens, "batch (x seq_len) -> (batch x) seq_len", x=8, seq_len=128
#     )
#     all_tokens_reshaped[:, 0] = model.tokenizer.bos_token_id
#     all_tokens_reshaped = all_tokens_reshaped[
#         torch.randperm(all_tokens_reshaped.shape[0])
#     ]
#     torch.save(all_tokens_reshaped, "/workspace/data/c4_code_2b_tokens_reshaped.pt")
# else:
#     # data = datasets.load_from_disk("/workspace/data/c4_code_tokenized_2b.hf")
#     all_tokens = torch.load("/workspace/data/owt_tensor.pt")
#     # all_tokens = all_tokens[: cfg["num_tokens"] // cfg["seq_len"]]
#     # all_tokens = shuffle_data(all_tokens)

### connor's dataset

In [ ]:
from datasets import load_from_disk

tokenized_data = load_from_disk("data/pile-lmsys-mix-1m-tokenized-gemma-2.hf")
tokenized_data # num_rows = 963,566

In [ ]:
len(tokenized_data[0]["input_ids"])

In [ ]:
963566*1024 # 986_691_584

### new dataset

- lmsys/lmsys-chat-1m
- monology/pile-uncopyrighted

In [1]:
import torch
from pprint import pprint
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from transformer_lens import utils, HookedTransformer

/home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
!pip install transformer_lens --quiet --upgrade

In [3]:
base_model_id = "Qwen/Qwen2.5-0.5B"
# instruct_model_id = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer_chat_template = "qwen-2.5"
tokens_max_len = 1024

DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}

In [4]:
model = HookedTransformer.from_pretrained(base_model_id).to(DTYPES["fp32"])

n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab

print(f"n_layers: {n_layers}")
print(f"d_model: {d_model}")
print(f"n_heads: {n_heads}")
print(f"d_head: {d_head}")
print(f"d_mlp: {d_mlp}")
print(f"d_vocab: {d_vocab}")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loaded pretrained model Qwen/Qwen2.5-0.5B into HookedTransformer
Changing model dtype to torch.float32
n_layers: 24
d_model: 896
n_heads: 14
d_head: 64
d_mlp: 4864
d_vocab: 151936


#### lmsys

In [5]:
# in the blog but i didn't use this
# """User: {instruction}
# Assistant: {completion}
# """

In [6]:
lmsys = load_dataset(
    "lmsys/lmsys-chat-1m", 
    # split="train[:500000]"
    split="train"
)
lmsys

Dataset({
    features: ['conversation_id', 'model', 'conversation', 'turn', 'language', 'openai_moderation', 'redacted'],
    num_rows: 1000000
})

In [7]:
tokenizer = get_chat_template(
    model.tokenizer,
    chat_template=tokenizer_chat_template,
)

def formatting_prompts_func(examples):
    convos = examples["conversation"]
    texts = [
        tokenizer.apply_chat_template(
            convo, 
            tokenize=False, 
            add_generation_prompt=False
        ) 
        for convo in convos
    ]
    return { "text" : texts }

lmsys = lmsys.map(formatting_prompts_func, batched=True)
lmsys

Model does not have a padding token! Will use pad_token = <|vision_pad|>.


Map: 100%|██████████| 1000000/1000000 [01:28<00:00, 11243.37 examples/s]


Dataset({
    features: ['conversation_id', 'model', 'conversation', 'turn', 'language', 'openai_moderation', 'redacted', 'text'],
    num_rows: 1000000
})

In [8]:
tokenized_lmsys = utils.tokenize_and_concatenate(
    dataset=lmsys, 
    tokenizer=model.tokenizer,
    max_length=tokens_max_len,
    add_bos_token=False,
)
tokenized_lmsys

Map (num_proc=10): 100%|██████████| 1000000/1000000 [04:06<00:00, 4059.63 examples/s]


Dataset({
    features: ['tokens'],
    num_rows: 521429
})

#### pile

In [9]:
# pile = load_dataset("monology/pile-uncopyrighted", split="train[:500000]", trust_remote_code=True)
pile = load_dataset(
    "NeelNanda/pile-10k", 
    split="train", 
    trust_remote_code=True
)
print(pile)

Dataset({
    features: ['text', 'meta'],
    num_rows: 10000
})


In [10]:
print(pile[0]["text"])

It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.

There’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.

Concept

Working over the theme was probably one of the hardest tasks I had to face.

Originally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.

In the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge. But is it a game?



In [11]:
tokenized_pile = utils.tokenize_and_concatenate(
    dataset=pile, 
    tokenizer=model.tokenizer,
    max_length=tokens_max_len,
    add_bos_token=False,
)
tokenized_pile


Map (num_proc=10): 100%|██████████| 10000/10000 [00:11<00:00, 859.01 examples/s]


Dataset({
    features: ['tokens'],
    num_rows: 15036
})

#### concatenate, save to disk and push to hub

In [12]:
from datasets import concatenate_datasets

tokenized_data = concatenate_datasets([tokenized_lmsys, tokenized_pile])

tokenized_data = tokenized_data.shuffle(42)
tokenized_data

Dataset({
    features: ['tokens'],
    num_rows: 536465
})

In [13]:
tokenized_data = tokenized_data.rename_column(
    original_column_name="tokens",
    new_column_name="input_ids",
)

In [14]:
tokenized_data

Dataset({
    features: ['input_ids'],
    num_rows: 536465
})

In [15]:
# save tokens to disk

dataset_path = "data/pile-lmsys-mix-500k-tokenized-qwen2x5"
# dataset_path = "data/pile-lmsys-mix-300k-tokenized-qwen2x5"

tokenized_data.save_to_disk(f"{dataset_path}.hf")

tokenized_data.set_format(type="torch", columns=["input_ids"])
all_tokens = tokenized_data["input_ids"]
print(all_tokens.shape)

torch.save(all_tokens, f"{dataset_path}.pt")
print(f"Saved tokens to disk")

Saving the dataset (9/9 shards): 100%|██████████| 536465/536465 [00:04<00:00, 112533.98 examples/s]


torch.Size([536465, 1024])
Saved tokens to disk


In [ ]:
# tokens = 277699*1024  # 284_363_776
# 536465*1024 # 549_340_160

549340160

In [17]:
# push data to hub
dataset_id = "aastha6/pile-lmsys-mix-500k-tokenized-qwen2.5"
# dataset_id = "aastha6/pile-lmsys-mix-300k-tokenized-qwen2.5"
tokenized_data.push_to_hub(dataset_id, private=False)

Uploading the dataset shards: 100%|██████████| 9/9 [00:55<00:00,  6.12s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/aastha6/pile-lmsys-mix-500k-tokenized-qwen2.5/commit/7833593f7d52e72994ea5c4263feedf653a709e2', commit_message='Upload dataset', commit_description='', oid='7833593f7d52e72994ea5c4263feedf653a709e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/aastha6/pile-lmsys-mix-500k-tokenized-qwen2.5', endpoint='https://huggingface.co', repo_type='dataset', repo_id='aastha6/pile-lmsys-mix-500k-tokenized-qwen2.5'), pr_revision=None, pr_num=None)

In [ ]:
# CommitInfo(commit_url='https://huggingface.co/datasets/aastha6/pile-lmsys-mix-300k-tokenized-qwen2.5/commit/7a12968c732ed4e4519e03b4bafc1f5ac31efba5', commit_message='Upload dataset', commit_description='', oid='7a12968c732ed4e4519e03b4bafc1f5ac31efba5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/aastha6/pile-lmsys-mix-300k-tokenized-qwen2.5', endpoint='https://huggingface.co', repo_type='dataset', repo_id='aastha6/pile-lmsys-mix-300k-tokenized-qwen2.5'), pr_revision=None, pr_num=None)